# Extract generater model

In [3]:
!gsutil ls gs://chair-generation/models

gs://chair-generation/models/
gs://chair-generation/models/edges2chair_ikea/


In [9]:
!gsutil cp -r gs://chair-generation/models/edges2chair_ikea/ ../models

Copying gs://chair-generation/models/edges2chair_ikea/checkpoint...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565351093.tensorflow-20190806-153451...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565698804.tensorflow-20190806-153451...
Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1565767058.tensorflow-20190806-153451...
| [4 files][261.4 MiB/261.4 MiB]   26.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://chair-generation/models/edges2chair_ikea/events.out.tfevents.1566206506.chair-demo-instance...
Copying gs://chair-generation/models/edges2chair_ikea/graph.pbtxt...            
Copying gs://chair-generation/models/edges2chair_ikea/model-31506

In [10]:
!python ../pix2pix.py \
  --mode export \
  --output_dir ../export/edges2chair_ikea/ \
  --checkpoint ../models/edges2chair_ikea/

/Users/huijie/.pyenv/versions/3.5.7/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huijie/.pyenv/versions/3.5.7/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huijie/.pyenv/versions/3.5.7/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huijie/.pyenv/versions/3.5.7/lib/pyt

In [11]:
!gsutil cp -r ../export/edges2chair_ikea/ gs://chair-generation/export

Copying file://../export/edges2chair_ikea/checkpoint [Content-Type=application/octet-stream]...
Copying file://../export/edges2chair_ikea/options.json [Content-Type=application/json]...
Copying file://../export/edges2chair_ikea/export.index [Content-Type=application/octet-stream]...
Copying file://../export/edges2chair_ikea/export.meta [Content-Type=application/octet-stream]...
- [4 files][377.3 KiB/377.3 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../export/edges2chair_ikea/export.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by

# Deploy model

In [13]:
!gcloud ai-platform models create "chair_generation"

Created ml engine model [projects/chair-search-demo/models/chair_generation].


In [16]:
MODEL_DIR="gs://chair-generation/export/edges2chair_ikea/export"
VERSION_NAME="v001"
MODEL_NAME="chair_generation"
FRAMEWORK="tensorflow"

!gcloud ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.14 \
  --framework $FRAMEWORK \
  --python-version=3.5

Creating version (this might take a few minutes)......done.                    


# Test

In [17]:
!gcloud ai-platform predict --model $MODEL_NAME  \
                   --version $VERSION_NAME \
                   --json-instances instance.json

OUTPUT_BYTES
{u'b64': u'...'}


In [31]:
import googleapiclient 
from oauth2client.client import GoogleCredentials
import json

In [25]:
def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1',)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
import base64

# Open and read image as bitstring
input_image = open("../data/sketch/cpvr2016/train/1.png", "rb").read()
print("Raw bitstring: " + str(input_image[:10]) + " ... " + str(input_image[-10:]))

# Encode image in b64
encoded_input_string = base64.b64encode(input_image)
input_string = encoded_input_string.decode()
request_data = {'image_bytes': {"b64": input_string}}

output = predict_json("chair-search-demo", "chair_generation", request_data, version=None)